In [13]:
import torch

batch_size = 1

input_image = torch.randn(batch_size, 3, 32, 32)

conv_weight11 = torch.randn(64, 3, 3, 3)
conv_bias11 = torch.randn(64)
conv_weight12 = torch.randn(64, 64, 3, 3) # (num kernel, depth, i, j)
conv_bias12 = torch.randn(64)

conv_weight21 = torch.randn(128, 64, 3, 3)
conv_bias21 = torch.randn(128)
conv_weight22 = torch.randn(128, 128, 3, 3)
conv_bias22 = torch.randn(128)

conv_weight31 = torch.randn(256, 128, 3, 3)
conv_bias31 = torch.randn(256)
conv_weight32 = torch.randn(256, 256, 3, 3)
conv_bias32 = torch.randn(256)
conv_weight33 = torch.randn(256, 256, 3, 3)
conv_bias33 = torch.randn(256)

conv_weight41 = torch.randn(512, 256, 3, 3)
conv_bias41 = torch.randn(512)
conv_weight42 = torch.randn(512, 512, 3, 3)
conv_bias42 = torch.randn(512)
conv_weight43 = torch.randn(512, 512, 3, 3)
conv_bias43 = torch.randn(512)

conv_weight51 = torch.randn(512, 512, 3, 3)
conv_bias51 = torch.randn(512)
conv_weight52 = torch.randn(512, 512, 3, 3)
conv_bias52 = torch.randn(512)
conv_weight53 = torch.randn(512, 512, 3, 3)
conv_bias53 = torch.randn(512)

fc_weight1 = torch.randn(4096, 512*7*7)
fc_bias1 = torch.randn(4096)
fc_weight2 = torch.randn(4096, 4096)
fc_bias2 = torch.randn(4096)
fc_weight3 = torch.randn(10, 4096)
fc_bias3 = torch.randn(10)



In [4]:

print(input_image.shape)
print(input_image.shape[0])
print(conv_weight32.shape)
print(len(conv_weight11))
print(conv_bias11.shape)

torch.Size([1, 3, 32, 32])
1
torch.Size([256, 256, 3, 3])
64
torch.Size([64])


In [5]:
# zero padding logic before each convolution
def zero_padding(input_image):
    batch_size = input_image.shape[0]
    in_channel = input_image.shape[1]
    num_row = input_image.shape[2]
    num_col = input_image.shape[3]

    padding_image = torch.zeros(batch_size, in_channel, num_row + 2, num_col + 2)

    for bc in range(batch_size): # take each in_feature
        for ch in range(in_channel): # take each in channel
            for row in range(num_row):
                for col in range(num_col):
                    padding_image[bc][ch][row + 1][col + 1] = input_image[bc][ch][row][col]
                    
    return padding_image

In [6]:
def conv2d_numpy(input_image, conv_weight, conv_bias):
    input_image = zero_padding(input_image)
    
    batch_size = input_image.shape[0]
    in_channel = input_image.shape[1]
    num_row = input_image.shape[2] - 2
    num_col = input_image.shape[3] - 2

    num_kernel = conv_weight.shape[0]
    conv_image = torch.zeros(batch_size, num_kernel, num_row, num_col) 
    for bc in range(batch_size): # take each in_feature
        for nk in range(num_kernel): # take each kernel
            for row in range(num_row): # take each row
                for col in range(num_col): # take each col
                    sum = 0
                    for ch in range(in_channel): # take each in channel
                        for i in range(3):
                            for j in range(3):
                                sum += input_image[bc][ch][row + i][col + j] * conv_weight[nk][ch][i][j]
                                
                    sum += conv_bias[nk]
                    conv_image[bc][nk][row][col] = sum
                    
    return conv_image
                            
                    

In [7]:
# ReLU logic after each convolution
def relu(input_image):
    batch_size = input_image.shape[0]
    in_channel = input_image.shape[1]
    num_row = input_image.shape[2]
    num_col = input_image.shape[3]

    relu_image = torch.zeros(batch_size, in_channel, num_row, num_col)

    for bc in range(batch_size): # take each in_feature
        for ch in range(in_channel): # take each in channel
            for row in range(num_row):
                for col in range(num_col):
                    relu_image[bc][ch][row][col] = input_image[bc][ch][row][col] if (input_image[bc][ch][row][col] > 0) else 0
                    
    return relu_image

In [8]:
# MaxPooling logic, pool_size=2, stride=2
def maxpool(input_image):
    batch_size = input_image.shape[0]
    in_channel = input_image.shape[1]
    num_row = input_image.shape[2]
    num_col = input_image.shape[3]

    pool_image = torch.zeros(batch_size, in_channel, num_row / 2, num_col / 2)

    for bc in range(batch_size): # take each in_feature
        for ch in range(in_channel): # take each in channel
            for row in range(0, num_row, 2):
                for col in range(0, num_col, 2):
                    max_value = 0
                    for i in range(2):
                        for j in range(2):
                            if max_value < input_image[bc][ch][row + i][col + j]:
                                max_value = input_image[bc][ch][row + i][col + j]
                    
                    pool_image[bc][ch][row / 2][col / 2] = max_value
                    
    return pool_image

In [9]:
def flatten(input_image):
    batch_size = input_image.shape[0]
    in_channel = input_image.shape[1]
    num_row = input_image.shape[2]
    num_col = input_image.shape[3]

    flatten_image = torch.zeros(batch_size, in_channel*num_row*num_col)

    for bc in range(batch_size): # take each in_feature
        for ch in range(in_channel): # take each in channel
            for row in range(num_row):
                for col in range(num_col):
                    flatten_image[bc][in_channel*num_row*num_col + row*num_col + col] = input_image[bc][ch][row][col]
                    
    return flatten_image
    

In [10]:
def fc_layer(in_image, fc_weight, fc_bias):
    batch_size = in_image.shape[0]
    in_size = in_image.shape[1] # length
    
    out_size = fc_weight.shape[0]
    
    out_image = torch.zeros(batch_size, out_size)
    
    for bc in range(batch_size):
        for i in range(out_size):
            sum = 0
            for j in range(in_size):
                sum += in_image[bc][j] * fc_weight[i][j]
        
            out_image[bc][i] = sum + fc_bias[i]
            
    
    return out_image

In [14]:
# Convolution test
# input_image : (1, 3, 32, 32)
tensor = conv2d_numpy(input_image, conv_weight11, conv_bias11)


'\ntensor = relu(tensor)\ntensor = conv2d_numpy(tensor, conv_weight12, conv_bias12)\ntensor = relu(tensor)\ntensor = maxpool(tensor)\n\n# Layer 2\ntensor = conv2d_numpy(tensor, conv_weight21, conv_bias21)\ntensor = relu(tensor)\ntensor = conv2d_numpy(tensor, conv_weight22, conv_bias22)\ntensor = relu(tensor)\ntensor = maxpool(tensor)\n\n# Layer 3\ntensor = conv2d_numpy(tensor, conv_weight31, conv_bias31)\ntensor = relu(tensor)\ntensor = conv2d_numpy(tensor, conv_weight32, conv_bias32)\ntensor = relu(tensor)\ntensor = conv2d_numpy(tensor, conv_weight33, conv_bias33)\ntensor = relu(tensor)\ntensor = maxpool(tensor)\n\n# Layer 4\ntensor = conv2d_numpy(tensor, conv_weight41, conv_bias41)\ntensor = relu(tensor)\ntensor = conv2d_numpy(tensor, conv_weight42, conv_bias42)\ntensor = relu(tensor)\ntensor = conv2d_numpy(tensor, conv_weight43, conv_bias43)\ntensor = relu(tensor)\ntensor = maxpool(tensor)\n\n# Layer 5\ntensor = conv2d_numpy(tensor, conv_weight51, conv_bias51)\ntensor = relu(tensor)

In [15]:
# Convolution test
print(tensor.shape)

torch.Size([1, 64, 32, 32])


In [ ]:
# Layer 1
tensor = conv2d_numpy(input_image, conv_weight11, conv_bias11)
tensor = relu(tensor)
tensor = conv2d_numpy(tensor, conv_weight12, conv_bias12)
tensor = relu(tensor)
tensor = maxpool(tensor)

# Layer 2
tensor = conv2d_numpy(tensor, conv_weight21, conv_bias21)
tensor = relu(tensor)
tensor = conv2d_numpy(tensor, conv_weight22, conv_bias22)
tensor = relu(tensor)
tensor = maxpool(tensor)

# Layer 3
tensor = conv2d_numpy(tensor, conv_weight31, conv_bias31)
tensor = relu(tensor)
tensor = conv2d_numpy(tensor, conv_weight32, conv_bias32)
tensor = relu(tensor)
tensor = conv2d_numpy(tensor, conv_weight33, conv_bias33)
tensor = relu(tensor)
tensor = maxpool(tensor)

# Layer 4
tensor = conv2d_numpy(tensor, conv_weight41, conv_bias41)
tensor = relu(tensor)
tensor = conv2d_numpy(tensor, conv_weight42, conv_bias42)
tensor = relu(tensor)
tensor = conv2d_numpy(tensor, conv_weight43, conv_bias43)
tensor = relu(tensor)
tensor = maxpool(tensor)

# Layer 5
tensor = conv2d_numpy(tensor, conv_weight51, conv_bias51)
tensor = relu(tensor)
tensor = conv2d_numpy(tensor, conv_weight52, conv_bias52)
tensor = relu(tensor)
tensor = conv2d_numpy(tensor, conv_weight53, conv_bias53)
tensor = relu(tensor)
tensor = maxpool(tensor)


In [ ]:
# fc connected
tensor = flatten(tensor)
tensor = fc_layer(tensor, fc_weight1, fc_bias1)
tensor = relu(tensor)
tensor = fc_layer(tensor, fc_weight2, fc_bias2)
tensor = relu(tensor)
tensor = fc_layer(tensor, fc_weight3, fc_bias3)

print(tensor.shape)

: 